In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Police violence

In [98]:
PV_data = pd.read_excel('./data/MPVDatasetDownload.xlsx')
PV_data.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),MPV ID,Fatal Encounters ID
0,Pedro Martinez,66,Male,Hispanic,NaN,2020-11-18,NaN,Tyler,TX,NaN,...,Allegedly Armed,unknown weapon,other,not fleeing,no,6321.0,NaN,NaN,NaN,NaN
1,James Horton,42,Male,NaN,NaN,2020-11-17,NaN,Hart County,KY,NaN,...,Allegedly Armed,gun,attack,other,no,6322.0,NaN,NaN,NaN,NaN
2,Dustin James Acosta,28,Male,White,NaN,2020-11-17,NaN,St. Augustine,FL,NaN,...,Allegedly Armed,gun,attack,NaN,No,6320.0,NaN,NaN,NaN,NaN
3,Javon Brice,39,Male,NaN,NaN,2020-11-17,NaN,La Vergne,TN,NaN,...,Allegedly Armed,gun,attack,car,no,6319.0,NaN,NaN,NaN,NaN
4,Joshua D. Evans,32,Male,NaN,NaN,2020-11-16,NaN,Batesville,IN,NaN,...,Allegedly Armed,gun,attack,not fleeing,no,6314.0,NaN,NaN,NaN,NaN


In [99]:
PV_data.columns

Index(['Victim's name', 'Victim's age', 'Victim's gender', 'Victim's race',
       'URL of image of victim', 'Date of Incident (month/day/year)',
       'Street Address of Incident', 'City', 'State', 'Zipcode', 'County',
       'Agency responsible for death', 'ORI Agency Identifier (if available)',
       'Cause of death',
       'A brief description of the circumstances surrounding the death',
       'Official disposition of death (justified or other)',
       'Criminal Charges?',
       'Link to news article or photo of official document',
       'Symptoms of mental illness?', 'Unarmed/Did Not Have an Actual Weapon',
       'Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)',
       'Alleged Threat Level (Source: WaPo)', 'Fleeing (Source: WaPo)',
       'Body Camera (Source: WaPo)', 'WaPo ID (If included in WaPo database)',
       'Off-Duty Killing?',
       'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-co

In [100]:
PV_data.drop(["Victim's name",
           "URL of image of victim", "Street Address of Incident", "Zipcode", "County", "ORI Agency Identifier (if available)",
           "Agency responsible for death",
           "A brief description of the circumstances surrounding the death",
           "Official disposition of death (justified or other)",
           "Link to news article or photo of official document", "Symptoms of mental illness?",
           "Alleged Threat Level (Source: WaPo)", #3968 are attack, the rest are undetermined/other
           "Body Camera (Source: WaPo)", "WaPo ID (If included in WaPo database)",
           "Off-Duty Killing?", #239/8629 are not NaN
           "Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )",
           "MPV ID", "Fatal Encounters ID"], 
          inplace=True, axis=1)

In [101]:
PV_data = PV_data.rename(
    columns={
        "Victim's age": 'victim_age',
        "Victim's gender": 'victim_gender',
        "Victim's race": 'victim_race',
        "Date of Incident (month/day/year)": 'date',
        "Cause of death": 'cause_of_death',                                
        "Criminal Charges?": 'criminal_charges',
        "Unarmed/Did Not Have an Actual Weapon": 'victim_armed',
        "Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)": 'alleged_weapon',
        "Alleged Threat Level (Source: WaPo)": 'alleged_threat_level',
        "Fleeing (Source: WaPo)": 'victim_fleeing'
    })

PV_data.head()

,victim_age,victim_gender,victim_race,date,City,State,cause_of_death,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
0,66,Male,Hispanic,2020-11-18,Tyler,TX,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing
1,42,Male,NaN,2020-11-17,Hart County,KY,Gunshot,No known charges,Allegedly Armed,gun,other
2,28,Male,White,2020-11-17,St. Augustine,FL,Gunshot,No known charges,Allegedly Armed,gun,NaN
3,39,Male,NaN,2020-11-17,La Vergne,TN,Gunshot,No known charges,Allegedly Armed,gun,car
4,32,Male,NaN,2020-11-16,Batesville,IN,Gunshot,No known charges,Allegedly Armed,gun,not fleeing


In [102]:
PV_data['victim_race'].unique()

array(['Hispanic', nan, 'White', 'Unknown race', 'Black', 'Unknown Race',
       'Native American', 'unknown race', 'Asian', 'Pacific Islander'],
      dtype=object)

In [103]:
PV_data['victim_race'] = PV_data['victim_race'].fillna('unknown race') #Replace NaN values with "unknown race"
PV_data['victim_race'] = PV_data['victim_race'].str.lower()
PV_data['victim_race'].unique()

array(['hispanic', 'unknown race', 'white', 'black', 'native american',
       'asian', 'pacific islander'], dtype=object)

In [104]:
PV_data['criminal_charges'].unique()

array(['No known charges', 'Charged with a crime',
       'Charged, Convicted, Sentenced to 7 years in prison',
       'Charged, Acquitted',
       'Charged, Convicted, Sentenced to 1 year in prison and 4 years probation',
       'Charged, Charges Dropped',
       'Charged, Convicted, Sentenced to 10 years in prison',
       'Charged, Convicted', 'Charged with a crime, Acquitted',
       'Charged, Plead No Contest, Sentenced to 10 years probation',
       'Charged, Convicted, Sentenced to 6 years in prison',
       'Charged, Convicted of reckless homicide, sentenced to 1 year in prison',
       'Charged, Convicted, Sentenced to 14 years in prison',
       'Charged, Convicted, Sentenced to 5 years in prison',
       'Charged, Convicted, Sentenced to 40 years in prison',
       'Charged, Convicted, Sentenced to 25 years in prison',
       'Charged, Convicted, Sentenced to 3 months in jail',
       'Charged, Mistrial',
       'Charged, Convicted, Sentenced to 30 years in prison',
       '

In [105]:
PV_data.loc[PV_data['criminal_charges'] != "No known charges", 'criminal_charges'] = "Charged"
PV_data['criminal_charges'].unique()

array(['No known charges', 'Charged'], dtype=object)

In [106]:
PV_data['victim_armed'].sort_values(na_position='first')

0       Allegedly Armed
5223    Allegedly Armed
5222    Allegedly Armed
5221    Allegedly Armed
5218    Allegedly Armed
             ...       
7677            Vehicle
7685            Vehicle
2534            Vehicle
7630            Vehicle
3546            Vehicle
Name: victim_armed, Length: 8629, dtype: object

In [107]:
PV_data['victim_armed'].unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle',
       'Unarmed/Did Not Have an Actual Weapon'], dtype=object)

In [108]:
PV_data.loc[PV_data['victim_armed'] == "Unarmed/Did Not Have an Actual Weapon", 'victim_armed'] = "Unarmed"
PV_data['victim_armed'].unique()

array(['Allegedly Armed', 'Unclear', 'Vehicle', 'Unarmed'], dtype=object)

In [109]:
PV_data['victim_fleeing'].unique()

array(['not fleeing', 'other', nan, 'car', 'foot', 'Not fleeing', 'Foot',
       'Car', 'Not Fleeing', 'Other'], dtype=object)

In [110]:
#We probably want to get rid of the NaNs here too
#PV_data['victim_fleeing'] = PV_data['victim_fleeing'].fillna('unknown') 
PV_data['victim_fleeing']=PV_data['victim_fleeing'].str.lower()
PV_data['victim_fleeing'].unique()

array(['not fleeing', 'other', nan, 'car', 'foot'], dtype=object)

In [111]:
PV_data.query('victim_age == "Unknown"')
#266 victims are of "unknown" age

,victim_age,victim_gender,victim_race,date,City,State,cause_of_death,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
222,Unknown,Male,unknown race,2020-08-29,Loma,CO,Gunshot,No known charges,Allegedly Armed,gun,not fleeing
233,Unknown,NaN,unknown race,2020-08-24,Albuquerque,NM,Gunshot,No known charges,Allegedly Armed,gun,NaN
246,Unknown,Male,unknown race,2020-08-20,Brooksville,FL,Gunshot,No known charges,Allegedly Armed,gun,NaN
252,Unknown,Male,white,2020-08-18,Bay City,TX,Gunshot,No known charges,Allegedly Armed,gun,foot
253,Unknown,Male,hispanic,2020-08-17,Bakersfield,CA,Gunshot,No known charges,Allegedly Armed,knife,car
...,...,...,...,...,...,...,...,...,...,...,...
8208,Unknown,Male,unknown race,2013-05-22,Philadelphia,PA,Gunshot,No known charges,Allegedly Armed,gun,NaN
8258,Unknown,Male,unknown race,2013-05-08,Miami,FL,Gunshot,No known charges,Unclear,undetermined,NaN
8266,Unknown,Male,unknown race,2013-05-04,Miami,FL,Gunshot,No known charges,Allegedly Armed,gun,NaN
8315,Unknown,Male,unknown race,2013-04-22,Longwood,FL,Gunshot,No known charges,Allegedly Armed,gun,NaN


In [112]:
PV_data.loc[PV_data['victim_age'] == "Unknown",  'victim_age'] = np.NaN

In [113]:
PV_data['victim_age'].unique()

array([66, 42, 28, 39, 32, nan, 50, 35, 38, 40, 19, 18, 16, 52, 37, 61,
       29, 27, 36, 22, 25, 33, 58, 56, 34, 43, 26, 60, 30, 55, 20, 59, 44,
       21, 64, 53, 62, 63, 45, 48, 46, 41, 57, 47, 23, 51, 24, 31, 88, 54,
       17, 67, 73, 49, 70, 68, 81, 65, 69, 82, 83, 75, 15, 76, 74, 77, 80,
       14, 71, 6, 78, 72, 84, 89, 1, '40s', 79, 91, 13, 86, 10, 12, 95, 5,
       93, 7, 107, 87], dtype=object)

In [114]:
PV_data.query('victim_age == "40s"')

,victim_age,victim_gender,victim_race,date,City,State,cause_of_death,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
3502,40s,Male,black,2017-09-28,Oakland,CA,Taser,No known charges,Unarmed,no object,foot


In [115]:
PV_data.loc[PV_data['victim_age'] == "40s", 'victim_age'] = 40 #Only one record is of age '40s', so we allow ourselves to change that into 40

In [116]:
PV_data['date'] = pd.DatetimeIndex(PV_data['date']).year #changes dates to years

In [118]:
PV_data.alleged_weapon.unique()

array(['unknown weapon', 'gun', 'undetermined', 'vehicle', 'knife',
       'gun, vehicle', 'curtain rod', 'sharp object', 'machete',
       'no object', 'hatchet', 'shovel', 'Airsoft pistol', 'toy weapon',
       'pepper spray', 'metal pipe', 'bottle', 'gun and knife', 'sword',
       'baseball bat', 'Taser', 'pellet gun', 'vehicle and gun', 'hammer',
       'rock', 'samurai sword', 'gun and car', 'screwdriver',
       'windshield wiper', 'gun and vehicle', 'pick-axe', 'tire iron',
       'BB gun', 'scissors', 'meat cleaver', 'chair', 'chain',
       'car, knife and mace', 'vehicle and machete', 'box cutter', 'ax',
       'crowbar', 'baseball bat and knife', 'straight edge razor',
       'baton', 'air pistol', 'wasp spray', 'BB gun and vehicle',
       'piece of wood', 'garden tool', 'barstool', 'walking stick',
       'wrench', 'beer bottle', 'flag pole', 'lawn mower blade',
       'crossbow', 'metal object', 'bow and arrow', 'lamp', 'glass shard',
       'incendiary device', 'pipe', 

In [122]:
import re #RegEx

#We are interested in guns
PV_data['alleged_weapon'] = PV_data['alleged_weapon'].replace(to_replace ='.*gun.*', value = 'gun', regex = True)

Result of the cleaning

In [123]:
PV_data.head()

,victim_age,victim_gender,victim_race,date,City,State,cause_of_death,criminal_charges,victim_armed,alleged_weapon,victim_fleeing
0,66,Male,hispanic,2020,Tyler,TX,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing
1,42,Male,unknown race,2020,Hart County,KY,Gunshot,No known charges,Allegedly Armed,gun,other
2,28,Male,white,2020,St. Augustine,FL,Gunshot,No known charges,Allegedly Armed,gun,NaN
3,39,Male,unknown race,2020,La Vergne,TN,Gunshot,No known charges,Allegedly Armed,gun,car
4,32,Male,unknown race,2020,Batesville,IN,Gunshot,No known charges,Allegedly Armed,gun,not fleeing


Exporting the daraframe to csv to be used in other notebooks

In [125]:
PV_data.to_csv('./data/police_violence.csv')

***

# 2013-2019 killings per state

In [48]:
data_per_state = pd.read_excel('./data/MPVDatasetDownload.xlsx', sheet_name="2013-2019 Killings by State")
data_per_state.head()

,State,State Abbreviation,Total Population,Black Population,Hispanic Population,Native American Population,Asian Population,Pacific Islander Population,White Population,Other Population,...,Rate (Hispanic People),Rate (Native American People),Rate (Asian People),Rate (Pacific Islanders),Rate (White People),Rate (All People),Disparity in Rate,Black-White Disparity,Hispanic-White Disparity,Native American-White Disparity
0,Alabama,AL,4864680.0,1285737.0,203146.0,23243.0,63936.0,1521.0,3196730.0,90367.0,...,0.000000,0.000000,2.234377,0.000000,3.306951,4.052535,1.453111,1.780730,0.000000,0.000000
1,Alaska,AK,738516.0,22817.0,51186.0,103506.0,45617.0,8544.0,450754.0,56092.0,...,2.790942,16.562187,0.000000,0.000000,5.704727,7.737525,3.236692,4.390041,0.489233,2.903239
2,Arizona,AZ,6946685.0,286614.0,2163312.0,271946.0,222477.0,12523.0,3825886.0,163927.0,...,7.594176,7.354401,0.000000,0.000000,5.302227,7.012595,2.203370,2.914124,1.432262,1.387040
3,Arkansas,AR,2990671.0,458536.0,219052.0,17342.0,43441.0,7877.0,2173849.0,70574.0,...,1.956483,0.000000,3.288533,0.000000,3.942973,4.967829,1.944122,2.449438,0.496195,0.000000
4,California,CA,39148760.0,2164519.0,15221577.0,138427.0,5525439.0,138911.0,14695836.0,1264051.0,...,4.570580,7.224024,1.137595,9.255669,3.275952,4.302271,2.868695,3.767425,1.395191,2.205168


In [49]:
data_per_state.columns

Index(['State', 'State Abbreviation', 'Total Population', 'Black Population',
       'Hispanic Population', 'Native American Population', 'Asian Population',
       'Pacific Islander Population', 'White Population', 'Other Population',
       '% African-American', '% Victims Black', 'Disparity',
       '# Black people killed', '# Hispanic people killed',
       '# Native American people killed', '# Asian people killed',
       '# Pacific Islanders killed', '# White people killed',
       '# Unknown Race people killed', '# People Killed',
       'Rate (Black People)', 'Rate (Hispanic People)',
       'Rate (Native American People)', 'Rate (Asian People)',
       'Rate (Pacific Islanders)', 'Rate (White People)', 'Rate (All People)',
       'Disparity in Rate', 'Black-White Disparity',
       'Hispanic-White Disparity', 'Native American-White Disparity'],
      dtype='object')

In [50]:
data_per_state.drop(["State", "Native American Population", 'Asian Population',
           'Pacific Islander Population', 'White Population', 'Other Population',
           '% African-American',
           '# Native American people killed', '# Asian people killed',
           '# Pacific Islanders killed', '# White people killed',
           '# Unknown Race people killed',
           'Rate (Native American People)', 'Rate (Asian People)',
           'Rate (Pacific Islanders)',
           'Native American-White Disparity'], 
            inplace=True, axis=1)

In [51]:
data_per_state = data_per_state.rename(
    columns={
        "State Abbreviation": "State"
    })

data_per_state.head()

,State,Total Population,Black Population,Hispanic Population,% Victims Black,Disparity,# Black people killed,# Hispanic people killed,# People Killed,Rate (Black People),Rate (Hispanic People),Rate (White People),Rate (All People),Disparity in Rate,Black-White Disparity,Hispanic-White Disparity
0,AL,4864680.0,1285737.0,203146.0,0.384058,0.119758,53.0,NaN,138.0,5.888785,0.000000,3.306951,4.052535,1.453111,1.780730,0.000000
1,AK,738516.0,22817.0,51186.0,0.100000,0.069104,4.0,1.0,40.0,25.043983,2.790942,5.704727,7.737525,3.236692,4.390041,0.489233
2,AZ,6946685.0,286614.0,2163312.0,0.090909,0.049650,31.0,115.0,341.0,15.451344,7.594176,5.302227,7.012595,2.203370,2.914124,1.432262
3,AR,2990671.0,458536.0,219052.0,0.298077,0.144755,31.0,3.0,104.0,9.658067,1.956483,3.942973,4.967829,1.944122,2.449438,0.496195
4,CA,39148760.0,2164519.0,15221577.0,0.158609,0.103319,187.0,487.0,1179.0,12.341904,4.570580,3.275952,4.302271,2.868695,3.767425,1.395191


In [52]:
data_per_state = data_per_state.rename(
    columns={
        'Total Population': "total_pop",
        'Black Population': "black_pop",
        'Hispanic Population': "hispanic_pop",
        '# Black people killed': "black_killings",
        '# Hispanic people killed': "hispanic_killings",
        '# People Killed': "total_killings",
        'Rate (Black People)': "black_rate",
        'Rate (Hispanic People)': "hispanic_rate", 
        'Rate (White People)': "white_rate", 
        'Rate (All People)': "total_rate",
        
    })

In [53]:
data_per_state.columns

Index(['State', 'total_pop', 'black_pop', 'hispanic_pop', '% Victims Black',
       'Disparity', 'black_killings', 'hispanic_killings', 'total_killings',
       'black_rate', 'hispanic_rate', 'white_rate', 'total_rate',
       'Disparity in Rate', 'Black-White Disparity',
       'Hispanic-White Disparity'],
      dtype='object')

Result of the cleaning

In [55]:
data_per_state.head()

,State,total_pop,black_pop,hispanic_pop,% Victims Black,Disparity,black_killings,hispanic_killings,total_killings,black_rate,hispanic_rate,white_rate,total_rate,Disparity in Rate,Black-White Disparity,Hispanic-White Disparity
0,AL,4864680.0,1285737.0,203146.0,0.384058,0.119758,53.0,NaN,138.0,5.888785,0.000000,3.306951,4.052535,1.453111,1.780730,0.000000
1,AK,738516.0,22817.0,51186.0,0.100000,0.069104,4.0,1.0,40.0,25.043983,2.790942,5.704727,7.737525,3.236692,4.390041,0.489233
2,AZ,6946685.0,286614.0,2163312.0,0.090909,0.049650,31.0,115.0,341.0,15.451344,7.594176,5.302227,7.012595,2.203370,2.914124,1.432262
3,AR,2990671.0,458536.0,219052.0,0.298077,0.144755,31.0,3.0,104.0,9.658067,1.956483,3.942973,4.967829,1.944122,2.449438,0.496195
4,CA,39148760.0,2164519.0,15221577.0,0.158609,0.103319,187.0,487.0,1179.0,12.341904,4.570580,3.275952,4.302271,2.868695,3.767425,1.395191


Exporting the dataframe to a csv file

In [56]:
data_per_state.to_csv('./data/killings.csv')

***

# ranking of states by gun laws strength:

In [66]:
url = r'https://giffords.org/lawcenter/resources/scorecard/#rankings'
tables = pd.read_html(url) # Returns list of all tables on page
gunlaw_ranking = tables[0] # Select table of interest

In [67]:
gunlaw_ranking.head()

,Gun LawStrength(Ranked),State,2019Grade,Gun DeathRate(Ranked),Gun DeathRate(Per 100K)
0,38,Alabama,F,2,21.70
1,42,Alaska,F,7,20.74
2,45,Arizona,F,18,15.29
3,40,Arkansas,F,8,18.96
4,1,California,A,44,7.45


In [68]:
gunlaw_ranking = gunlaw_ranking.rename(
    columns={
        "Gun LawStrength(Ranked)": 'state_gunlaw_strength',
        "2019Grade": 'state_gunlaw_grade',
        "Gun DeathRate(Ranked)": 'state_gun_death_rate',
        "Gun DeathRate(Per 100K)": 'state_gun_death_rate_per_100K'
    })
gunlaw_ranking.head()

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,38,Alabama,F,2,21.70
1,42,Alaska,F,7,20.74
2,45,Arizona,F,18,15.29
3,40,Arkansas,F,8,18.96
4,1,California,A,44,7.45


In [69]:
#We probably should put this in a file
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [71]:
gunlaw_ranking = gunlaw_ranking.replace({"State": us_state_abbrev})

The final result of cleaning

In [72]:
gunlaw_ranking.head()

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,38,AL,F,2,21.70
1,42,AK,F,7,20.74
2,45,AZ,F,18,15.29
3,40,AR,F,8,18.96
4,1,CA,A,44,7.45


In [126]:
gunlaw_ranking.sort_values(by=['state_gunlaw_strength'])


,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
4,1,CA,A,44,7.45
29,2,NJ,A,46,4.75
6,3,CT,A-,45,4.91
31,4,NY,A-,47,4.03
10,5,HI,A-,48,4.03
19,6,MD,A-,33,11.61
20,7,MA,A-,49,3.46
12,8,IL,A-,36,10.78
38,9,RI,B+,50,3.28
46,10,WA,B+,38,10.32


Exporting the dataframe to csv

In [73]:
gunlaw_ranking.to_csv('./data/gunlaw_ranking.csv')

***